In [ ]:
# Cài thư viện nếu cần
!pip install pandas scikit-learn matplotlib seaborn --quiet


#Bước 1: Tải và đọc dữ liệu

In [ ]:
import pandas as pd
from google.colab import drive

# 1. Gắn Google Drive của bạn
# Khi chạy đoạn này, bạn sẽ được yêu cầu cấp quyền truy cập.
# Hãy làm theo hướng dẫn trong output.
drive.mount('/content/gdrive')

# 2. Đọc dữ liệu từ Google Drive
# Thay đổi đường dẫn dưới đây cho phù hợp với vị trí file của bạn trong Drive.
# Ví dụ: nếu file 'annonimized.csv' của bạn nằm trong thư mục 'My Drive/data/'
# thì đường dẫn sẽ là '/content/gdrive/MyDrive/data/annonimized.csv'

# 'annonimized.csv'
df_annonimized = pd.read_csv("/content/gdrive/MyDrive/study/kì 8/Máy học/Đồ án cuối kỳ/annonimized.csv")




#  'th-public.csv'
df_qt_public = pd.read_csv("/content/gdrive/MyDrive/study/kì 8/Máy học/Đồ án cuối kỳ/qt-public.csv")







Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Bước 2: Tiền xử lý dữ liệu


In [ ]:
import numpy as np

# Chuyển datetime
df_annonimized['created_at'] = pd.to_datetime(df_annonimized['created_at'], format='%m-%d %H:%M:%S', errors='coerce')
df_annonimized['updated_at'] = pd.to_datetime(df_annonimized['updated_at'], format='%m-%d %H:%M:%S', errors='coerce')


# Trích đặc trưng thời gian
df_annonimized['submit_hour'] = df_annonimized['created_at'].dt.hour
df_annonimized['submit_weekday'] = df_annonimized['created_at'].dt.weekday
df_annonimized['delay'] = (df_annonimized['updated_at'] - df_annonimized['created_at']).dt.total_seconds()

# Use raw 'concat('it001', username)' as 'mssv'
df_annonimized['mssv'] = df_annonimized["concat('it001', username)"]

# Encode categorical
df_annonimized['assignment_id'] = df_annonimized["concat('it001',`assignment_id`)"].astype('category').cat.codes
df_annonimized['problem_id'] = df_annonimized["concat('it001',`problem_id`)"].astype('category').cat.codes

# Đảm bảo số liệu
df_annonimized['percent_testcase_passed'] = df_annonimized['pre_score'].fillna(0) / 100 # Assuming pre_score is out of 10000
df_annonimized['delay_weight'] = df_annonimized['coefficient'].fillna(100) / 100 # Assuming coefficient is out of 100
df_annonimized['has_score'] = df_annonimized['status'] == 'SCORE'
df_annonimized['is_passed'] = df_annonimized['percent_testcase_passed'] == 100

# Preview
df_annonimized[['mssv', 'assignment_id', 'problem_id', 'percent_testcase_passed', 'submit_hour', 'submit_weekday', 'delay', 'delay_weight', 'has_score', 'is_passed']].head()

,mssv,assignment_id,problem_id,percent_testcase_passed,submit_hour,submit_weekday,delay,delay_weight,has_score,is_passed
0,ed9eaeb6a707f50154024b24d7efcb874a9795dd,116,208,0.0,8.0,1.0,294.0,1.0,True,False
1,ed9eaeb6a707f50154024b24d7efcb874a9795dd,116,208,0.0,8.0,1.0,10.0,1.0,True,False
2,ed9eaeb6a707f50154024b24d7efcb874a9795dd,116,208,100.0,8.0,1.0,9.0,1.0,True,True
3,ed9eaeb6a707f50154024b24d7efcb874a9795dd,116,335,100.0,8.0,1.0,9.0,1.0,True,True
4,ed9eaeb6a707f50154024b24d7efcb874a9795dd,116,212,100.0,9.0,1.0,10.0,1.0,True,True


#Bước 3: Trích đặc trưng cho từng sinh viên



In [ ]:
# Gom theo mssv để tạo 1 dòng/mssv
features = df_annonimized.groupby('mssv').agg({
    'percent_testcase_passed': ['mean', 'std', 'max'],
    'delay_weight': 'mean',
    'submit_hour': 'mean',
    'submit_weekday': 'nunique',
    'assignment_id': 'nunique',
    'problem_id': 'nunique',
    'delay': 'mean',
    'has_score': 'sum',
    'is_passed': 'sum'
})

# Đổi tên cột
features.columns = ['_'.join(col) for col in features.columns]
features.reset_index(inplace=True)

# Preview
features.head()

,mssv,percent_testcase_passed_mean,percent_testcase_passed_std,percent_testcase_passed_max,delay_weight_mean,submit_hour_mean,submit_weekday_nunique,assignment_id_nunique,problem_id_nunique,delay_mean,has_score_sum,is_passed_sum
0,00b6dd4fc7eb817e03708c532016ef30ce564a61,55.041497,45.414358,100.0,1.000000,4.401361,7,7,46,8195.428571,119,54
1,00bef8afee8f3c595d535c9c03c490cac1a4f021,54.885521,43.237609,100.0,1.000000,10.525097,7,9,78,-232304.548263,209,85
2,01122b3ef7e59b84189e65985305f575d6bdf83c,59.737538,41.699366,100.0,1.000000,10.964103,7,7,66,-103020.830769,175,59
3,0134f9f410c65ad0e8c2254a7e9288670e02a183,59.527600,46.403021,100.0,1.000000,8.610000,7,4,47,1514.090000,76,52
4,013de369c439ab0ead8aa7da64423aa395a8be39,64.744486,44.096659,100.0,0.971963,8.598131,4,8,52,184.037383,94,59


#Bước 4: Chuẩn bị nhãn (label)

In [ ]:
# Đọc điểm thực hành thật
df_score_th = pd.read_csv("/content/gdrive/MyDrive/study/kì 8/Máy học/Đồ án cuối kỳ/th-public.csv")  # file này có 2 cột: mssv, diem_thuc_hanh

# Đổi tên cột 'hash' thành 'mssv' trong df_score_th
df_score_th = df_score_th.rename(columns={'hash': 'mssv'})

# Ghép vào feature
df_merged_th = pd.merge(features, df_score_th, on='mssv')
df_merged_th.head()

,mssv,percent_testcase_passed_mean,percent_testcase_passed_std,percent_testcase_passed_max,delay_weight_mean,submit_hour_mean,submit_weekday_nunique,assignment_id_nunique,problem_id_nunique,delay_mean,has_score_sum,is_passed_sum,TH
0,00b6dd4fc7eb817e03708c532016ef30ce564a61,55.041497,45.414358,100.0,1.000000,4.401361,7,7,46,8195.428571,119,54,5
1,00bef8afee8f3c595d535c9c03c490cac1a4f021,54.885521,43.237609,100.0,1.000000,10.525097,7,9,78,-232304.548263,209,85,8.5
2,01122b3ef7e59b84189e65985305f575d6bdf83c,59.737538,41.699366,100.0,1.000000,10.964103,7,7,66,-103020.830769,175,59,7
3,013de369c439ab0ead8aa7da64423aa395a8be39,64.744486,44.096659,100.0,0.971963,8.598131,4,8,52,184.037383,94,59,10
4,014c59c6433fd764a0b08de6ffeb757eaf60aa73,52.400251,46.917106,100.0,1.000000,11.989950,7,9,90,-252938.427136,154,79,6


#Bước 5: Train/Test mô hình

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import numpy as np
import pandas as pd
import lightgbm as lgb

# Tách features và labels
X = df_merged_th.drop(columns=['mssv', 'TH'])
y = df_merged_th['TH']

# Chia train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Impute missing values with the mean of the training data
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_train.mean()) # Use mean from training data to avoid data leakage

# Convert y to numeric, coercing errors to NaN and then dropping NaN
y_train = pd.to_numeric(y_train, errors='coerce').dropna()
y_test = pd.to_numeric(y_test, errors='coerce').dropna()

# Ensure X_train and y_train have the same index after dropping NaNs in y_train
X_train = X_train.loc[y_train.index]
X_test = X_test.loc[y_test.index]


# Huấn luyện model using LightGBM
model = lgb.LGBMRegressor(random_state=42)
model.fit(X_train, y_train)

# Dự đoán và đánh giá
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f"R^2 Score (TH with LightGBM): {r2:.4f}")

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000707 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1247
[LightGBM] [Info] Number of data points in the train set: 602, number of used features: 10
[LightGBM] [Info] Start training from score 7.305648
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

#Bước 6: Dự đoán cho tất cả sinh viên

In [ ]:
# Dự đoán cho tất cả sinh viên có trong annonimized.csv
# Use df_annonimized to generate features for all students
features_all = df_annonimized.groupby('mssv').agg({
    'percent_testcase_passed': ['mean', 'std', 'max'],
    'delay_weight': 'mean',
    'submit_hour': 'mean',
    'submit_weekday': 'nunique',
    'assignment_id': 'nunique',
    'problem_id': 'nunique',
    'delay': 'mean',
    'has_score': 'sum',
    'is_passed': 'sum'
})
features_all.columns = ['_'.join(col) for col in features_all.columns]
features_all.reset_index(inplace=True)

# Apply the same numeric conversion and imputation as the training data
# Ensure X_train is available from the previous step (cell 7qrkVtddXNPb)
if 'X_train' in locals():
    X_all = features_all.drop(columns=['mssv'])

    # Align columns of X_all with X_train before imputation and prediction
    X_all = X_all.reindex(columns=X_train.columns, fill_value=0)

    # Impute missing values using the mean from the training data (X_train)
    X_all = X_all.fillna(X_train.mean())

    # Dự đoán using the trained model (named 'model' in cell 7qrkVtddXNPb)
    y_all_pred_th = model.predict(X_all)

    # Tạo file submission cho TH
    submission_th = pd.DataFrame({
        'mssv': features_all['mssv'],
        'diem_du_doan_th': y_all_pred_th
    })

    # Làm tròn nếu muốn
    submission_th['diem_du_doan_th'] = submission_th['diem_du_doan_th'].clip(0, 10).round(2)

    # Preview
    display(submission_th.head())

else:
    print("X_train not found. Please run the previous cell to train the model first.")

,mssv,diem_du_doan_th
0,00b6dd4fc7eb817e03708c532016ef30ce564a61,5.72
1,00bef8afee8f3c595d535c9c03c490cac1a4f021,8.32
2,01122b3ef7e59b84189e65985305f575d6bdf83c,7.61
3,0134f9f410c65ad0e8c2254a7e9288670e02a183,6.64
4,013de369c439ab0ead8aa7da64423aa395a8be39,9.22


#Bước 7: Xuất file để nộp

In [ ]:
from google.colab import files

# Xuất file
submission_th.to_csv("submission_thuc_hanh.csv", index=False,header=False)

# Tải về
files.download("submission_thuc_hanh.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Task
Hướng dẫn tôi các bước để cải thiện khả năng chính xác dự đoán điểm thực hành của sinh viên dựa trên tập dữ liệu "annonimized.csv".

## Khám phá các mô hình hồi quy khác

### Subtask:
Thử nghiệm các mô hình hồi quy khác như Random Forest, Gradient Boosting Machines (ví dụ: XGBoost, LightGBM - đã thử nhưng có thể tinh chỉnh), Support Vector Regression (SVR), Lasso, Ridge để xem mô hình nào cho kết quả tốt nhất trên tập dữ liệu này.


**Reasoning**:
Import necessary libraries and initialize different regression models, then train each model on the training data and make predictions on the test data.



In [41]:
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Lasso, Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Initialize models
rf_model = RandomForestRegressor(random_state=42)
xgb_model = XGBRegressor(random_state=42)
svr_model = SVR()
lasso_model = Lasso(random_state=42)
ridge_model = Ridge(random_state=42)

# Train and evaluate Random Forest
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)
print(f"Random Forest - MAE: {mae_rf:.4f}, MSE: {mse_rf:.4f}, R^2: {r2_rf:.4f}")

# Train and evaluate XGBoost
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)
print(f"XGBoost - MAE: {mae_xgb:.4f}, MSE: {mse_xgb:.4f}, R^2: {r2_xgb:.4f}")

# Train and evaluate SVR
svr_model.fit(X_train, y_train)
y_pred_svr = svr_model.predict(X_test)
mae_svr = mean_absolute_error(y_test, y_pred_svr)
mse_svr = mean_squared_error(y_test, y_pred_svr)
r2_svr = r2_score(y_test, y_pred_svr)
print(f"SVR - MAE: {mae_svr:.4f}, MSE: {mse_svr:.4f}, R^2: {r2_svr:.4f}")


# Train and evaluate Lasso
lasso_model.fit(X_train, y_train)
y_pred_lasso = lasso_model.predict(X_test)
mae_lasso = mean_absolute_error(y_test, y_pred_lasso)
mse_lasso = mean_squared_error(y_test, y_pred_lasso)
r2_lasso = r2_score(y_test, y_pred_lasso)
print(f"Lasso - MAE: {mae_lasso:.4f}, MSE: {mse_lasso:.4f}, R^2: {r2_lasso:.4f}")

# Train and evaluate Ridge
ridge_model.fit(X_train, y_train)
y_pred_ridge = ridge_model.predict(X_test)
mae_ridge = mean_absolute_error(y_test, y_pred_ridge)
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
r2_ridge = r2_score(y_test, y_pred_ridge)
print(f"Ridge - MAE: {mae_ridge:.4f}, MSE: {mse_ridge:.4f}, R^2: {r2_ridge:.4f}")

Random Forest - MAE: 1.4554, MSE: 3.5093, R^2: 0.3750
XGBoost - MAE: 1.4543, MSE: 3.5853, R^2: 0.3614
SVR - MAE: 1.9044, MSE: 6.1777, R^2: -0.1003
Lasso - MAE: 1.7117, MSE: 4.6236, R^2: 0.1765
Ridge - MAE: 1.6072, MSE: 4.3043, R^2: 0.2334


**Reasoning**:
The previous step trained and evaluated various regression models. The next logical step in the subtask is to perform hyperparameter tuning on the promising models to potentially improve their performance. Based on the R-squared scores, Random Forest and XGBoost show the best performance so far, followed by Ridge and Lasso. SVR performed poorly (negative R-squared), so tuning it is not as high a priority. I will start by tuning the Random Forest model as it achieved the highest R2.



In [42]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for Random Forest
param_grid_th = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize GridSearchCV for Random Forest
grid_search_th = GridSearchCV(estimator=rf_model, param_grid=param_grid_th,
                           scoring='r2', cv=3, n_jobs=-1, verbose=1)

# Perform grid search on the training data
grid_search_th.fit(X_train, y_train)

# Get the best parameters and best score
best_params_th = grid_search_th.best_params_
best_score_th = grid_search_th.best_score_

print(f"Best parameters for Random Forest (TH): {best_params_th}")
print(f"Best R^2 score for Random Forest (TH): {best_score_th:.4f}")

# Evaluate the tuned Random Forest model on the test set
tuned_rf_model_th = grid_search_th.best_estimator_
y_pred_tuned_th = tuned_rf_model_th.predict(X_test)
mae_tuned_th = mean_absolute_error(y_test, y_pred_tuned_th)
r2_tuned_th = r2_score(y_test, y_pred_tuned_th)

print(f"Tuned Random Forest (TH) - MAE: {mae_tuned_th:.4f}, R^2: {r2_tuned_th:.4f}")


Fitting 3 folds for each of 81 candidates, totalling 243 fits
Best parameters for Random Forest (TH): {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 300}
Best R^2 score for Random Forest (TH): 0.2829
Tuned Random Forest (TH) - MAE: 1.4528, R^2: 0.3729


**Reasoning**:
I have tuned the Random Forest model. The next step is to tune the XGBoost model, as it also showed good performance in the initial evaluation.



In [43]:
# Define the parameter grid for XGBoost
param_grid_xgb = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7]
}

# Initialize GridSearchCV for XGBoost
grid_search_xgb = GridSearchCV(estimator=xgb_model, param_grid=param_grid_xgb,
                           scoring='r2', cv=3, n_jobs=-1, verbose=1)

# Perform grid search on the training data
grid_search_xgb.fit(X_train, y_train)

# Get the best parameters and best score
best_params_xgb = grid_search_xgb.best_params_
best_score_xgb = grid_search_xgb.best_score_

print(f"Best parameters for XGBoost (TH): {best_params_xgb}")
print(f"Best R^2 score for XGBoost (TH): {best_score_xgb:.4f}")

# Evaluate the tuned XGBoost model on the test set
tuned_xgb_model_th = grid_search_xgb.best_estimator_
y_pred_tuned_xgb = tuned_xgb_model_th.predict(X_test)
mae_tuned_xgb = mean_absolute_error(y_test, y_pred_tuned_xgb)
r2_tuned_xgb = r2_score(y_test, y_pred_tuned_xgb)

print(f"Tuned XGBoost (TH) - MAE: {mae_tuned_xgb:.4f}, R^2: {r2_tuned_xgb:.4f}")

Fitting 3 folds for each of 27 candidates, totalling 81 fits
Best parameters for XGBoost (TH): {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 200}
Best R^2 score for XGBoost (TH): 0.2621
Tuned XGBoost (TH) - MAE: 1.5015, R^2: 0.3316


**Reasoning**:
I have tuned Random Forest and XGBoost. While Ridge and Lasso had lower initial R2 scores, they are simpler models and might still benefit from tuning. I will now tune the Ridge and Lasso models to see if their performance can be improved.



In [44]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso, Ridge

# Define parameter grids for Lasso and Ridge
param_grid_lasso = {
    'alpha': [0.1, 0.5, 1.0, 2.0, 5.0]
}

param_grid_ridge = {
    'alpha': [0.1, 0.5, 1.0, 2.0, 5.0]
}

# Initialize and perform GridSearchCV for Lasso
grid_search_lasso = GridSearchCV(estimator=Lasso(random_state=42), param_grid=param_grid_lasso,
                                 scoring='r2', cv=3, n_jobs=-1, verbose=1)
grid_search_lasso.fit(X_train, y_train)

best_params_lasso = grid_search_lasso.best_params_
best_score_lasso = grid_search_lasso.best_score_
print(f"Best parameters for Lasso (TH): {best_params_lasso}")
print(f"Best R^2 score for Lasso (TH): {best_score_lasso:.4f}")

tuned_lasso_model_th = grid_search_lasso.best_estimator_
y_pred_tuned_lasso = tuned_lasso_model_th.predict(X_test)
mae_tuned_lasso = mean_absolute_error(y_test, y_pred_tuned_lasso)
r2_tuned_lasso = r2_score(y_test, y_pred_tuned_lasso)
print(f"Tuned Lasso (TH) - MAE: {mae_tuned_lasso:.4f}, R^2: {r2_tuned_lasso:.4f}")

# Initialize and perform GridSearchCV for Ridge
grid_search_ridge = GridSearchCV(estimator=Ridge(random_state=42), param_grid=param_grid_ridge,
                                 scoring='r2', cv=3, n_jobs=-1, verbose=1)
grid_search_ridge.fit(X_train, y_train)

best_params_ridge = grid_search_ridge.best_params_
best_score_ridge = grid_search_ridge.best_score_
print(f"Best parameters for Ridge (TH): {best_params_ridge}")
print(f"Best R^2 score for Ridge (TH): {best_score_ridge:.4f}")

tuned_ridge_model_th = grid_search_ridge.best_estimator_
y_pred_tuned_ridge = tuned_ridge_model_th.predict(X_test)
mae_tuned_ridge = mean_absolute_error(y_test, y_pred_tuned_ridge)
r2_tuned_ridge = r2_score(y_test, y_pred_tuned_ridge)
print(f"Tuned Ridge (TH) - MAE: {mae_tuned_ridge:.4f}, R^2: {r2_tuned_ridge:.4f}")

Fitting 3 folds for each of 5 candidates, totalling 15 fits
Best parameters for Lasso (TH): {'alpha': 0.1}
Best R^2 score for Lasso (TH): 0.2187
Tuned Lasso (TH) - MAE: 1.6262, R^2: 0.2266
Fitting 3 folds for each of 5 candidates, totalling 15 fits
Best parameters for Ridge (TH): {'alpha': 0.1}
Best R^2 score for Ridge (TH): 0.2224
Tuned Ridge (TH) - MAE: 1.6092, R^2: 0.2300


## Dự đoán cho tất cả sinh viên (với mô hình tốt nhất)

### Subtask:
Sử dụng mô hình Random Forest đã được tinh chỉnh (được lưu trong biến `tuned_rf_model_th`) để dự đoán điểm thực hành cho tất cả sinh viên trong tập dữ liệu `annonimized.csv`.


**Reasoning**:
Using the tuned Random Forest model to predict the practical scores for all students and create the submission dataframe, then display the head of the submission dataframe.



In [45]:
# Use the tuned Random Forest model to predict on the entire dataset
y_all_pred_th = tuned_rf_model_th.predict(X_all)

# Create the submission DataFrame
submission_th = pd.DataFrame({
    'mssv': features_all['mssv'],
    'diem_du_doan_th': y_all_pred_th
})

# Round the predictions to 2 decimal places and clip values between 0 and 10
submission_th['diem_du_doan_th'] = submission_th['diem_du_doan_th'].clip(0, 10).round(2)

# Display the first 5 rows of the submission DataFrame
display(submission_th.head())

,mssv,diem_du_doan_th
0,00b6dd4fc7eb817e03708c532016ef30ce564a61,6.62
1,00bef8afee8f3c595d535c9c03c490cac1a4f021,8.21
2,01122b3ef7e59b84189e65985305f575d6bdf83c,7.55
3,0134f9f410c65ad0e8c2254a7e9288670e02a183,6.63
4,013de369c439ab0ead8aa7da64423aa395a8be39,8.54


## Xuất file dự đoán

### Subtask:
Tạo file CSV với kết quả dự đoán điểm thực hành cho tất cả sinh viên và tải về.


**Reasoning**:
Export the submission DataFrame to a CSV file and download it.



In [47]:
from google.colab import files

# Export the submission DataFrame to a CSV file without index and header
submission_th.to_csv("submission_thuc_hanh.csv", index=False, header=False)

# Download the generated CSV file
files.download("submission_thuc_hanh.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Summary:

### Data Analysis Key Findings

*   Trong các mô hình hồi quy ban đầu được thử nghiệm (Random Forest, XGBoost, SVR, Lasso, Ridge), Random Forest và XGBoost cho kết quả R-squared tốt nhất trên tập kiểm tra, lần lượt là 0.3750 và 0.3614. Mô hình SVR có hiệu suất kém đáng kể với R-squared âm.
*   Sau khi tinh chỉnh siêu tham số, mô hình Random Forest đạt R-squared là 0.3729 trên tập kiểm tra.
*   Mô hình XGBoost sau khi tinh chỉnh siêu tham số có R-squared là 0.3316 trên tập kiểm tra, thấp hơn so với mô hình XGBoost ban đầu và Random Forest đã tinh chỉnh.
*   Các mô hình tuyến tính như Lasso và Ridge sau khi tinh chỉnh siêu tham số có R-squared lần lượt là 0.2266 và 0.2300 trên tập kiểm tra, thấp hơn đáng kể so với các mô hình dựa trên cây.
*   Mô hình Random Forest đã tinh chỉnh (`tuned_rf_model_th`) được sử dụng để dự đoán điểm thực hành cho tất cả sinh viên trong tập dữ liệu gốc.
*   Các điểm dự đoán được làm tròn đến 2 chữ số thập phân và giới hạn trong khoảng từ 0 đến 10.
*   Kết quả dự đoán được lưu vào file CSV có tên "submission\_thuc\_hanh.csv" mà không có index và header.

### Insights or Next Steps

*   Mô hình Random Forest đã tinh chỉnh là mô hình tốt nhất trong số các mô hình được thử nghiệm cho nhiệm vụ dự đoán điểm thực hành dựa trên tập dữ liệu này.
*   Hiệu suất R-squared cao nhất đạt được là khoảng 0.37, cho thấy vẫn còn một lượng đáng kể phương sai trong điểm thực hành chưa được giải thích bởi các đặc trưng hiện có. Có thể cần bổ sung thêm các đặc trưng liên quan hoặc xem xét các kỹ thuật mô hình hóa phức tạp hơn để cải thiện độ chính xác dự đoán.
